# Haciendo mas inteligentes a las Redes Neuronales (LLMs)

Aqui veremos como hacer mas inteligentes a las redes neuronales solucionando algunos de los problemas vistos en las unidades anteriores.



In [ ]:
!pip install openai --upgrade --quiet
from openai import OpenAI
##########################################
api_key="INGRESA AQUI TU TOKEN DE OPENAI"
##########################################
if api_key[25:] == "OPENAI":
    from google.colab import userdata
    api_key = userdata.get('OPENAI_TOKEN')
# defaults to os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [ ]:
def generar_texto(prompt, model="gpt-4o"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # aleatoriedad de las respuestas del modelo
    )
    return response.choices[0].message.content

In [ ]:
# Opcional si no tienes cuenta en OpenAI
# Puedes utilizar un modelo pequeño con huggingface pipelines
# Solo quita las comilla triples al siguiente codigo:

"""
from transformers import pipeline
generator = pipeline('text-generation', model = 'Qwen/Qwen2-0.5B-Instruct')
def generar_texto(prompt):
    return generator(prompt, max_length = 30, num_return_sequences=1)[0]["generated_text"]
"""

In [ ]:
generar_texto("Hola")

In [ ]:
generar_texto("En que año estamos?")

In [ ]:
generar_texto("Mi nombre es Hernan")

In [ ]:
generar_texto("Cual es mi nombre?")

## Resolviendo el problema de la Memoria

Como hemos visto la red neuronal no tiene memoria solo recibe una entrada y de acuerdo a esa entrada genera un texto de salida. No recuerda entradas anteriores que le hayamos enviado. Para solucionar ese problema debemos nosotros ir guardando todo el historial de entradas y salidas que van ocurriendo en la conversacion y pasarle todo el historial en nuevas llamadas.

### Ejercicio 1:

Escribe un codigo en python para ayudar a la red neuronal a tener memoria.

*Ayuda:*

```python
historial = ""
prompt = "Hola como estas? Mi nombre es Hernan"
generar_texto(prompt)
historial = historial + prompt
...
```

In [ ]:
# Escribir aqui la solucion




In [ ]:
#@title Solucion Ejercicio 1 {display-mode:"form"}

historial = ""

prompt = "Mi nombre es Hernan"
response = generar_texto(historial + " " + prompt)
print("Respuesta 1:", response)
historial += f"User: {prompt}\nBot: {response}\n"

prompt = "Cual es mi nombre?"
response = generar_texto(historial + " " + prompt)
print("Respuesta 2:", response)
historial += f"User: {prompt}\nBot: {response}\n"

print("Historial:", historial)

## Resolviendo el problema del año

Como hemos visto la red neuronal ha sido entrenada y puede contestar de acuerdo a la informacion que ha sido almacenada en sus pesos (parametros w). Toda la informacion nueva que surja luego de su entrenamiento es informacion a la que la red neuronal no puede acceder a menos que la ayudemos.

### Ejercicio 2:

Escribe un codigo en python para ayudar a la red neuronal a saber que año es si alguien le pregunta por el año.

*Ayuda 1:*

```python
def generar_texto2(prompt):
    if prompt contiene una pregunta sobre en que año estamos:
       return datetime.now().year
    else:
       return generar_texto(prompt)
```

In [ ]:
# Escribir aqui la solucion




In [ ]:
#@title Ayuda 2 Ejercicio 2 {display-mode:"form"}

from datetime import datetime
def generar_texto2(prompt):
    if prompt == "En que año estamos?":
       return datetime.now().year
    else:
       return generar_texto(prompt)

print(generar_texto2("En que año estamos?"))
print(generar_texto2("En que año nos encontramos?"))

In [ ]:
#@title Solucion Ejercicio 2 {display-mode:"form"}

from datetime import datetime
def generar_texto3(prompt):

    new_prompt = f"""Define si la pregunta del final es sobre en que año estamos.
    Responde solo si o no con minusculas y sin agregar acentos ni signos de puntuacion, solo usa dos letras.

    Pregunta:
    {prompt}

    Tu respuesta:
    """

    pregunta_el_anio = generar_texto(new_prompt)

    print(pregunta_el_anio)

    if pregunta_el_anio == "si":
       return datetime.now().year
    else:
       return generar_texto(prompt)

print(generar_texto3("En que año estamos?"))
print(generar_texto3("En que año nos encontramos?"))
print(generar_texto3("Cuantos dedos tiene una mano?"))

## Contestar a partir de informacion de un pdf nuestro

Esta tecnica se llama Retrieval Augmented Generation y se conoce como RAG.

Utilizamos [pypdf2](https://pypdf2.readthedocs.io/en/3.x/) para leer el siguiente pdf sobre delfines:
- https://www.amc.edu.mx/revistaciencia/images/revista/70_3/PDF/10_70_3_1156_Delfin_L.pdf


In [ ]:
!pip install PyPDF2 -U -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00


In [ ]:
import requests
from PyPDF2 import PdfReader

url = "https://www.amc.edu.mx/revistaciencia/images/revista/70_3/PDF/10_70_3_1156_Delfin_L.pdf"
response = requests.get(url)

with open("delfines.pdf", "wb") as f:
  f.write(response.content)

reader = PdfReader("delfines.pdf")
page = reader.pages[0]
texto = page.extract_text()
print(texto[0:500])

In [ ]:
def generar_texto4(prompt):

    new_prompt = f"""Responde la pregunta del final utlizando la siguiente informacion:
    {texto}

    Pregunta a responder:
    {prompt}

    Tu respuesta:
    """

    return generar_texto(new_prompt)

In [ ]:
generar_texto4("Que porcentaje de diversidad de delfines viven en Mexico?")

### Ejercicio 3:

Utiliza esta tecnica para preguntarle al chat con un documento propio.

In [ ]:
# Escribir aqui la solucion




## Hacer que un LLM pueda buscar informacion en Internet

---



In [ ]:
!pip install exa-py -U -q

In [ ]:
from exa_py import Exa
from google.colab import userdata

exa = Exa(api_key=userdata.get('EXA_TOKEN'))

#https://docs.exa.ai/reference/search
result = exa.search_and_contents(
  "campeon eurocopa 2024",
  type="neural",
  use_autoprompt=True,
  num_results=10,
  text=True
)

In [ ]:
for i, r in enumerate(result.results):
    print(i, r.title, r.url)

In [ ]:
def generar_texto5(prompt):

    respuesta_final = ""

    new_prompt = f"""Trata de responder la pregunta entre parentesis.
    Si no sabes como responder entonces solamente responde "No se" y no agregues nada mas.
    ({prompt})
    """

    sabe_responder = generar_texto(new_prompt)

    print("Primera respuesta:", sabe_responder)

    if sabe_responder == "No se":

        result = exa.search_and_contents(
          prompt,
          type="keyword",
          use_autoprompt=True,
          num_results=1,
          text=True
        )

        print("Respuesta del buscador:", result.results[0].text[0:1000], "....... (este texto fue truncado)")

        new_prompt = f"""Responde la pregunta del final utlizando la siguiente informacion:
        {result.results[0].text[0:16000]}

        Pregunta a responder:
        {prompt}

        Tu respuesta:
        """

        respuesta_final = generar_texto(new_prompt)


    else:
        respuesta_final = generar_texto(prompt)

    print("Respuesta final:", respuesta_final)
    return respuesta_final

In [ ]:
generar_texto5("Quien gano la eurocopa 2024?");

## Crear nuestra propia interfaz para chatear con nuestro LLM

- https://www.gradio.app/guides/creating-a-chatbot-fast#customizing-your-chatbot

In [ ]:
!pip install --upgrade gradio -q

In [ ]:
import gradio as gr

def mi_funcion_de_generacion_de_texto(message, history):
  """
  message: a str representing the user's input.
  history: a list of list representing the conversations up until that point.
  Each inner list consists of two str representing a pair: [user input, bot response].
  """
  return generar_texto(message)

gr.ChatInterface(mi_funcion_de_generacion_de_texto).launch()

In [ ]:
import gradio as gr

def convert_history_to_text(history):
    text = ""
    for pair in history:
        user_message, bot_message = pair
        text += f"User: {user_message}\n"
        text += f"Bot: {bot_message}\n"

    return text

def mi_funcion_de_generacion_de_texto(message, history):
  """
  message: a str representing the user's input.
  history: a list of list representing the conversations up until that point. Each inner list consists of two str representing a pair: [user input, bot response].
  """
  prompt = convert_history_to_text(history) + f"User: {message}\nBot:"



  return generar_texto(prompt)

gr.ChatInterface(
    mi_funcion_de_generacion_de_texto,
    chatbot=gr.Chatbot(height=500),
    textbox=gr.Textbox(placeholder="Hazme una pregunta", container=False, scale=7),
    title="Mi Chat",
    #description="Ask Yes Man any question",
    theme="soft",
    #examples=["Hello", "Am I cool?", "Are tomatoes vegetables?"],
    cache_examples=True,
    retry_btn=None,
    undo_btn=None,
    clear_btn=None,
).launch()

### Ejercicio 4:

Crea tu propio chatbot y agregale alguna de las funcionalidades vistas anteriormente para hacerlo mas inteligente.

In [ ]:
# Escribir aqui la solucion




## Referencias:

- https://docs.exa.ai/reference/getting-started-with-python
- https://colab.research.google.com/drive/1Aj6bBptSHWxZO7GVG2RoWtQSEkpabuaF


## A continuacion:

- https://github.com/run-llama/llama_parse/blob/main/examples/multimodal/multimodal_report_generation_agent.ipynb